# STEP 3 — Rule-Based Customer Segmentation (Python)
Objective

Convert insights into business-usable segments

Implement exact rules given in the project brief

Create clean, binary flags (0/1)

Validate segment sizes and overlaps

## 3.1 Load EDA-Ready Data

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/marketing_data_eda_ready.csv")


## 3.2 Compute Thresholds Required for Segmentation

90th Percentile for High Spender

In [2]:
high_spend_threshold = df['Total_Spend'].quantile(0.90)
high_spend_threshold


np.float64(1564.0)

## 3.3 Implement Segments

#### 1️⃣ High Income Customers
Rule: Income > 75,000

In [3]:
df['High_Income'] = np.where(df['Income'] > 75000, 1, 0)

#### 2️⃣ Young Customers

Rule: Age < 30

In [4]:
df['Young_Customer'] = np.where(df['Age'] < 30, 1, 0)

#### 3️⃣ Campaign Responders

Rule: Response = 1

In [5]:
df['Campaign_Responder'] = df['Response']

#### 4️⃣ High Web Engagement

Rule: NumWebVisitsMonth > 5

In [6]:
df['High_Web_Engagement'] = np.where(df['NumWebVisitsMonth'] > 5, 1, 0)


#### 5️⃣ Family Customers

Rule: Children > 0

In [ ]:
df['Family_Customer'] = np.where(df['Children'] > 0, 1, 0)

#### 6️⃣ High Spenders

Rule: Total_Spend > 90th percentile

In [8]:
df['High_Spender'] = np.where(df['Total_Spend'] > high_spend_threshold, 1, 0)


## 3.4 Validate Segment Sizes

In [9]:
segment_cols = [
    'High_Income',
    'Young_Customer',
    'Campaign_Responder',
    'High_Web_Engagement',
    'Family_Customer',
    'High_Spender'
]

df[segment_cols].mean() * 100


High_Income            35.325000
Young_Customer          0.000000
Campaign_Responder     14.758929
High_Web_Engagement    53.517857
Family_Customer        68.253571
High_Spender            9.998214
dtype: float64

## Interpretation:

~10% customers are High Spenders (as expected)

A smaller but valuable portion are High Income

Campaign Responders align strongly with High Spenders and Web Engagement

## 3.5 Segment Performance Validation
### Response Rate by Segment

In [10]:
for col in segment_cols:
    response_rate = df[df[col] == 1]['Response'].mean() * 100
    print(f"{col}: {response_rate:.2f}% response rate")


High_Income: 23.54% response rate
Young_Customer: nan% response rate
Campaign_Responder: 100.00% response rate
High_Web_Engagement: 12.07% response rate
Family_Customer: 11.62% response rate
High_Spender: 26.04% response rate


## 3.6 Create Human-Readable Segment Labels

In [11]:
def customer_profile(row):
    if row['High_Spender'] == 1 and row['High_Web_Engagement'] == 1:
        return 'High Value Digital'
    elif row['High_Spender'] == 1:
        return 'High Value'
    elif row['High_Web_Engagement'] == 1:
        return 'Digitally Engaged'
    elif row['Family_Customer'] == 1:
        return 'Family Oriented'
    else:
        return 'General'

df['Customer_Segment'] = df.apply(customer_profile, axis=1)


## 3.7 Segment Summary Table

In [12]:
segment_summary = df.groupby('Customer_Segment').agg(
    customers=('ID', 'count'),
    avg_income=('Income', 'mean'),
    avg_spend=('Total_Spend', 'mean'),
    response_rate=('Response', 'mean')
).reset_index()

segment_summary['response_rate'] *= 100
segment_summary


,Customer_Segment,customers,avg_income,avg_spend,response_rate
0,Digitally Engaged,27939,46064.840603,405.062994,11.174344
1,Family Oriented,14292,55723.134561,506.510285,13.210188
2,General,8170,77030.953914,766.250428,21.995104
3,High Value,3568,90079.674482,1982.163677,26.961883
4,High Value Digital,2031,83619.122661,1954.707041,24.421467


## 3.8 Save Segmented Dataset

In [13]:
df.to_csv("../data/marketing_data_segmented.csv", index=False)
